This script generates the 2 remaining files required for input:
- True positive genes (nucleotide fasta)
- protein alignment (amino acid fasta)
IMPORTANTLY, these 2 files are created from non-overlapping sequences!


And also moves the corresponding files to new directories so that coverage tables can be generated from them.

In the first cell specify:
- BGC type (This name must stay constant throughout the scripts)
- select_neg_genomes, i.e. the amount of negative genomes to be transferred to the neg_genomes directory
- select_neg_genomes, i.e. the amount of positive genomes to be transferred to the pos_genomes directory and to generate the tp_genes file from (the surplus amount will be used to generate the protein alignment from)
- pos_isolation_source_filter, if these terms are found in the isolation_source column of the positive samples in the summary file, they will be scored higher in a scoring column, i.e. samples from a known and desired isolation source will be used preferentially.
- neg_isolation_source_filter, accordingly
- avoid_list. These terms are scored with a 0, end at the bottom of the table, and will be picked last. This is useful when an uncommon gene is searched for and more, and/or more tenuous isolation sources have been allowed during download

In [21]:
BGC_type = 'Rieske'
select_neg_genomes = 10
select_pos_genomes = 3

avoid_list = ['', 'isolation_source not annotated']
#these are identical to first script, but don't have to be
pos_isolation_source_filter =  ['marine', 'sea', 'sponge', 'ocean', 'porifera', 'seafloor','sediment', 'water', 'tidal', 'coral', 'reef', 'coast', 'ship', 'fish', 'aquaculture', 'atlantic', 'pacific', 'mediterranean', 'baltic', 'pond']
neg_isolation_source_filter = ['marine', 'sea', 'sponge', 'ocean', 'porifera', 'seafloor', 'sediment', 'water', 'tidal', 'coral', 'reef', 'coast', 'ship', 'fish', 'aquaculture', 'atlantic', 'pacific', 'mediterranean', 'baltic', 'pond']

In [22]:
import os
from os import listdir, mkdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
import random
import glob
import warnings

In [23]:
def makedir(dirpath):
    if os.path.isdir(dirpath):
        print(dirpath,'exists already')
    else:
        print('Making', dirpath)    
        os.mkdir(dirpath)

        
# Defining paths for required directory structure for input and output files relative to parent directory
parent_dir='/nesi/project/vuw03285/'
BGC_path=os.path.join(parent_dir, BGC_type)
neg_genomes_path=os.path.join(BGC_path, 'base_genomes/temp_neg_genomes')
pos_genomes_path=os.path.join(BGC_path, 'base_genomes/temp_pos_genomes')
output_neg_path=os.path.join(BGC_path, 'base_genomes/neg_genomes')
output_pos_path=os.path.join(BGC_path, 'base_genomes/pos_genomes')


# Calling function to make directories if they don't exist yet
makedir(output_neg_path)
makedir(output_pos_path)

os.chdir(BGC_path)

/nesi/project/vuw03285/Rieske/base_genomes/neg_genomes exists already
/nesi/project/vuw03285/Rieske/base_genomes/pos_genomes exists already


In [ ]:
%%capture cap --no-stderr
# Generating a report file for this particular script
print('\nBGC_type =', BGC_type)
print('\nselect_neg_genomes =', select_neg_genomes)
print('\nselect_pos_genomes =', select_pos_genomes)
print('\navoid_list =', avoid_list)
print('\nneg_isolation_source_filter =', neg_isolation_source_filter)
print('\npos_isolation_source_filter =', pos_isolation_source_filter, '\n')
with open(BGC_path+'/'+'report_generate_tp.txt', 'w') as f:
    f.write(str(cap))

In [24]:
# load summary table into data frame () output from 1.)
summary_file = pd.read_csv('summary.tsv', sep='\t')

Change order of tables to prioritize samples that have an isolation source

In [25]:
warnings.filterwarnings('ignore')

#filter positives and drop all duplicate protein sequences originating from different organisms
pos_mask = (summary_file['dir'] == '+')
pos_df = summary_file[pos_mask]
pos_df.drop_duplicates(subset='protein_id', keep=False, inplace=True)


#filter negatives
neg_mask = (summary_file['dir'] == '-')
neg_df = summary_file[neg_mask]

#scoring words in isolation source so as to preferentially pick samples with chosen isolation sources

def custom_sorting(source,isolation_source_filter):
    score = 1
    if isolation_source_filter=='pos':
        for word in pos_isolation_source_filter:
            if word in source:
                score +=1
        for word in avoid_list:
            if source == word:
                score=0
    elif isolation_source_filter=='neg':
        for word in neg_isolation_source_filter:
            if word in source:
                score +=1
        for word in avoid_list:
            if source == word:
                score=0
    return score


pos_df['scoring_column'] = pos_df.apply(lambda x: custom_sorting(x['isolation_source'],'pos'),axis=1)
neg_df['scoring_column'] = neg_df.apply(lambda x: custom_sorting(x['isolation_source'],'neg'),axis=1)

pos_df.sort_values(by=['scoring_column'], axis=0, ascending=False, inplace=True)
neg_df.sort_values(by=['scoring_column'], axis=0, ascending=False, inplace=True)

In [26]:
#Split positive genomes into 2 bins, one goes towards tp-genes and is the pos-genomes used for synthesising metagenomes
#the other one constitutes a source of protein sequences for alignment as an input file

# Genomes selected in such a way that they are from the top of the pre-sorted pos_df
unique_pos_df = pos_df.drop_duplicates(subset='assembly', inplace=False)
selected_tp_genomes = list(unique_pos_df.iloc[:,1])[0:select_pos_genomes]
remaining_pos_genomes = list(unique_pos_df.iloc[:,1])[select_pos_genomes:]

#select genomes and isolate GCF number from them, move selected tp genomes to final pos_genomes directory
for genome in selected_tp_genomes:
    print('moving positive', genome, 'to', output_pos_path)
    !mv "{pos_genomes_path}"/"{genome}"* "{output_pos_path}"
    
#generate dataframe containing all tp-genomes and all the tp-genes contained in it
filtered_pos_df = pos_df[pos_df['assembly'].isin(selected_tp_genomes)]
remaining_pos_df = pos_df[~pos_df['assembly'].isin(selected_tp_genomes)]

#isolate all the headers and transfer them to the selected_tp_genes file
full_header_list = []
for i in range(0,len(filtered_pos_df)):
    full_header=str('>')+filtered_pos_df.iloc[i,1]+str('_')+filtered_pos_df.iloc[i,3]+str('_')+filtered_pos_df.iloc[i,5]
    full_header_list.append(full_header)

# generate fasta file with selected tp genes found in the selected genomes
print('generating selected_tp_genes.fasta')
with open(BGC_path+'/'+BGC_type+'_tp_genes.fasta') as fh:
    lines=fh.readlines()
    for i in range(0,len(lines)):
        for j in range(0,len(full_header_list)):
            if full_header_list[j] in lines[i]:
                with open(BGC_path+'/'+BGC_type+'_selected_tp_genes.fasta', 'a') as outfile:
                    outfile.write(lines[i]+lines[i+1])    
    
# transfer all amino acid sequences that are not part of the tp-genomes to a fasta file
print('generating selected_tp_aa.fasta')
with open(BGC_path+'/'+BGC_type+'_selected_tp_aa.fasta', 'a') as outfile:
    for i in range(0,len(remaining_pos_df)):
        fasta_header=str('>')+remaining_pos_df.iloc[i,1]+str('_')+remaining_pos_df.iloc[i,3]+str('_')+remaining_pos_df.iloc[i,5]+'\n'
        sequence = remaining_pos_df.iloc[i,6][2:-2]+'\n'
        outfile.write(fasta_header)
        outfile.write(sequence)
        
        
#Move selected neg genome files to different location
unique_neg_df = neg_df.drop_duplicates(subset='assembly', inplace=False)
selected_neg_genomes = list(unique_neg_df.iloc[:,1])[0:select_neg_genomes]

for genome in selected_neg_genomes:
    print('moving negative', genome, 'to', output_neg_path)
    !mv "{neg_genomes_path}"/"{genome}"* "{output_neg_path}"

    
print('Done')

moving positive GCF_009363655.1 to /nesi/project/vuw03285/Rieske/base_genomes/pos_genomes
moving positive GCF_002116735.1 to /nesi/project/vuw03285/Rieske/base_genomes/pos_genomes
moving positive GCF_013459415.1 to /nesi/project/vuw03285/Rieske/base_genomes/pos_genomes
generating selected_tp_genes.fasta
generating selected_tp_aa.fasta
moving negative GCF_000195575.1 to /nesi/project/vuw03285/Rieske/base_genomes/neg_genomes
moving negative GCF_011404475.1 to /nesi/project/vuw03285/Rieske/base_genomes/neg_genomes
moving negative GCF_002213725.1 to /nesi/project/vuw03285/Rieske/base_genomes/neg_genomes
moving negative GCF_002982175.1 to /nesi/project/vuw03285/Rieske/base_genomes/neg_genomes
moving negative GCF_013267615.1 to /nesi/project/vuw03285/Rieske/base_genomes/neg_genomes
moving negative GCF_005161825.1 to /nesi/project/vuw03285/Rieske/base_genomes/neg_genomes
moving negative GCF_003354405.1 to /nesi/project/vuw03285/Rieske/base_genomes/neg_genomes
moving negative GCF_001702435.1 t

In [27]:
!module load MUSCLE/3.8.1551
#not sure why full path is required?
!/opt/nesi/CS400_centos7_bdw/MUSCLE/3.8.1551/bin/muscle -in "{BGC_path}"/"{BGC_type}"_selected_tp_aa.fasta -out "{BGC_path}"/"{BGC_type}"_selected_tp_alignment.fasta
!module unload MUSCLE/3.8.1551


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

Rieske_selected_tp_aa 38 seqs, lengths min 74, max 1122, avg 338
00:00:00    19 MB(-2%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00    19 MB(-2%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00    36 MB(-3%)  Iter   1  100.00%  Align node       
00:00:00    36 MB(-3%)  Iter   1  100.00%  Root alignment
00:00:01    37 MB(-3%)  Iter   2  100.00%  Refine tree   
00:00:01    37 MB(-3%)  Iter   2  100.00%  Root alignment
00:00:01    37 MB(-3%)  Iter   2  100.00%  Root alignment
00:00:01    37 MB(-3%)  Iter   3  100.00%  Refine biparts
00:00:02    37 MB(-3%)  Iter   4  100.00%  Refine biparts
00:00:02    37 MB(-3%)  Iter   5  100.00%  Refine biparts
00:00:02    37 MB(-3%)  Iter   5  100.00%  Refine biparts
00:00:02    37 MB(-3%)  Iter   6  100.00%  Refine biparts
00:00:02    37 MB(-3%)  Iter   7  100.00%  Refine bipart